# Scraping many pages + Using Selenium

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Researching mine information

## Preparation 

### When you search for information on a specific mine, what URL should Selenium visit first?

- *TIP: the answer is NOT `https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp`*

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
driver = webdriver.Chrome()

In [2]:
url='https://arlweb.msha.gov/drs/drshome.htm'
driver.get(url)

### How can you identify the text field we're going to type the Mine ID into?

Selenium can find elements by:

- name
- Class
- ID
- CSS selector (**ASK ME WHAT THIS IS** if you don't know)
- XPath (**ASK ME WHAT THIS IS** because you definitely don't know)
- Link text
- Partial link text

So in other words, what's unique about this element?

In [3]:
mineID_Field = driver.find_element_by_name("MineId")
#searching by XPath was not getting the right input field

### How can you identify the search button we're going to click, or the form we're going to submit?

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

In [4]:
xPath_SearchButton = '//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input'
searchButton = driver.find_element_by_xpath(xPath_SearchButton)

### Use Selenium to search using the mine ID `3901432`. Get me the operator's name by scraping.

- *TIP: You can find elements/text using Selenium, or use BeautifulSoup with `doc = BeautifulSoup(driver.page_source)`*

In [5]:
mineID_Field.click()
mineID_Field.send_keys("3901432")
searchButton.click()

In [6]:
doc = BeautifulSoup(driver.page_source, "html.parser")
for line in doc.find_all():
    if line.text == "Operator:":
        print(line.find_next("b").text)

Krueger Brothers Gravel & Dirt


# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [7]:
df = pd.read_csv('mines-subset.csv')

### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them?

In [8]:
#one starts with a 0, IDs have to be changed to a string
df = pd.read_csv('mines-subset.csv', converters={'id': str})

### Scrape the operator's name for each of those mines and print it

- *TIP: use .apply and a function*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook *

In [9]:
def scrape_page(row):
    driver.get(url)
    mineID_Field = driver.find_element_by_name("MineId")
    xPath_SearchButton = '//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input'
    searchButton = driver.find_element_by_xpath(xPath_SearchButton)
    mineID_Field.click()
    mineID_Field.send_keys(row['id'])
    searchButton.click()
    doc = BeautifulSoup(driver.page_source, "html.parser")
    mine_name = ""
    for line in doc.find_all():
        if line.text == "Operator:":
            mine_name = line.find_next("b").text
            print(mine_name)

df.apply(scrape_page, axis=1)

Dirt Works
Holley Dirt Company, Inc
M.R. Dirt Inc.


0    None
1    None
2    None
dtype: object

### Scrape the operator's name and save it into a new column

- *TIP: Use .apply and a function*
- *TIP: Remember to use `return`*

In [10]:
def scrape_page(row):
    driver.get(url)
    mineID_Field = driver.find_element_by_name("MineId")
    xPath_SearchButton = '//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input'
    searchButton = driver.find_element_by_xpath(xPath_SearchButton)
    mineID_Field.click()
    mineID_Field.send_keys(row['id'])
    searchButton.click()
    doc = BeautifulSoup(driver.page_source, "html.parser")
    mine_name = ""
    for line in doc.find_all():
        if line.text == "Operator:":
            mine_name = line.find_next("b").text
    return mine_name

df['mine_name'] = df.apply(scrape_page, axis=1)
df

,id,mine_name
0,4104757,Dirt Works
1,0801306,"Holley Dirt Company, Inc"
2,3609931,M.R. Dirt Inc.


# Researching mine violations

Read the very top again to remember how to find mine violations

### When you search for a mine's violations, what URL is Selenium going to start on?

- *TIP: `requests` can send form data to load in the middle of a bunch of steps, but Selenium has to start at the beginning

In [11]:
# same URL as before to search for a specific mine first
url='https://arlweb.msha.gov/drs/drshome.htm'

### When you're searching for violations from the Mine Information page, how are you going to identify the "Beginning Date" field?

In [12]:
# beginningDate_field = driver.find_element_by_name("BDate")

### When you're searching for violations from the Mine Information page, how are you going to identify the "Violations" button?

In [13]:
# violations_button = driver.find_element_by_xpath("//input[(@type='radio') and (@value = 'Violations*')]")

### When you're searching for violations from the Mine Information page, how are you going to identify the form or the button to click to get a list of the violations?

In [14]:
# submit_button = driver.find_element_by_xpath("//input[(@alt='Submit Request')]")

### Using the mine ID `3901432`, scrape all of their violations since 1/1/1995

**Save this into a CSV called `3901432-violations.csv`.** This CSV must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

**Tips:**

- *TIP: It's probably worth it to print them all first, then save them to a CSV once you know it's all working.*
- *TIP: You'll use the parent pattern - get the ROWS first (tr), then loop through and get the TABLE CELLS (td)*

In [15]:
this_MineID = "3901432"
driver.get(url)
mineID_Field = driver.find_element_by_name("MineId")
xPath_SearchButton = '//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input'
searchButton = driver.find_element_by_xpath(xPath_SearchButton)
mineID_Field.click()
mineID_Field.send_keys(this_MineID)
searchButton.click()

In [16]:
beginningDate_field = driver.find_element_by_name("BDate")
violations_button = driver.find_element_by_xpath("//input[(@type='radio') and (@value = 'Violations*')]")
submit_button = driver.find_element_by_xpath("//input[(@alt='Submit Request')]")

In [17]:
beginningDate_field.send_keys("1/1/1995")
violations_button.click()
submit_button.click()

In [18]:
doc = BeautifulSoup(driver.page_source, "html.parser")
violations_list = []
violations = doc.find_all("tr",attrs={'class':'drsviols'})

for violation in violations:
    data = violation.find_all("td")
    if data:
        current = {}
        current["Citation number"] = data[2].text.strip()
        current["Case number"] = data[3].text.strip()
        current["Standard violated"] = data[10].find_all('font')[2].text.strip()
        current["Link to standard"] = data[10].find('a')['href']
        current["Proposed penalty"] = data[11].text.strip()
        current["Amount paid to date"] = data[14].text.strip()
        violations_list.append(current)

In [19]:
df = pd.DataFrame(violations_list)
df

,Amount paid to date,Case number,Citation number,Link to standard,Proposed penalty,Standard violated
0,100.00,000361866,8750964,http://www.gpo.gov/fdsys/pkg/CFR-2014-title30-...,100.00,56.18010
1,100.00,000260865,6426438,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.00,56.4101
2,100.00,000260865,6426439,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.00,56.4201(a)(2)
3,100.00,000260865,6588189,http://www.gpo.gov/fdsys/pkg/CFR-2011-title30-...,100.00,56.14200
4,100.00,000238554,6588210,http://www.gpo.gov/fdsys/pkg/CFR-2010-title30-...,100.00,50.30(a)
5,100.00,000188398,6328074,http://www.gpo.gov/fdsys/pkg/CFR-2009-title30-...,100.00,56.11003
6,55.00,390143205501,7916124,http://www.gpo.gov/fdsys/pkg/CFR-1998-title30-...,55.00,56.14132(a)
7,55.00,390143205501,7916121,http://www.gpo.gov/fdsys/pkg/CFR-1998-title30-...,55.00,56.18002
8,55.00,390143205501,7916118,http://www.gpo.gov/fdsys/pkg/CFR-1998-title30-...,55.00,56.12028
9,55.00,390143205501,7916116,http://www.gpo.gov/fdsys/pkg/CFR-1998-title30-...,55.00,50.30


In [20]:
filename = this_MineID+"-violations.csv"
df.to_csv(filename, index=False)

# Using .apply to save mine data for SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [21]:
df_new = pd.read_csv('mines-subset.csv', converters={'id': str})
df_new

,id
0,4104757
1,0801306
2,3609931


### Scrape the violations for each mine

**Save each mine's violations into separate CSV files.** Each CSV file must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

Make sure you are saving them into **separate files.** It might be nice to name them after the mine id.

- *TIP: Use .apply for this*
- *TIP: Print out the ID before you start scraping. That way you can take that ID and search manually to see if there is anything weird about the results.*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook 
- *TIP: It's probably worth it to print the fields first, then save them to a CSV once you know it's all working.*

In [22]:
def save_violations_csv(row):
    url='https://arlweb.msha.gov/drs/drshome.htm'
    violations_MineID = row['id']
    driver.get(url)
    
    mineID_Field = driver.find_element_by_name("MineId")
    xPath_SearchButton = '//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input'
    searchButton = driver.find_element_by_xpath(xPath_SearchButton)
    mineID_Field.click()
    mineID_Field.send_keys(violations_MineID)
    searchButton.click()
    
    beginningDate_field = driver.find_element_by_name("BDate")
    violations_button = driver.find_element_by_xpath("//input[(@type='radio') and (@value = 'Violations*')]")
    submit_button = driver.find_element_by_xpath("//input[(@alt='Submit Request')]")
    
    beginningDate_field.send_keys("1/1/1995")
    violations_button.click()
    submit_button.click()
    
    doc = BeautifulSoup(driver.page_source, "html.parser")
    violations_list = []
    violations = []
    violations = doc.find_all("tr",attrs={'class':'drsviols'})
    if violations:
        for violation in violations:
            data = []
            data = violation.find_all("td")
            if data:
                current = {}
                current["Citation number"] = data[2].text.strip()
                
                #exception handling first -- not all columns filled
                if len(data)<13:
                    current["Proposed penalty"] = ""
                    current["Amount paid to date"] = ""
                        
                    #exception 1 "Not assessed yet"
                    if len(data[10].find_all('font'))>2:
                        current["Case number"] = ""
                        current["Standard violated"] = data[10].find_all('font')[2].text.strip()
                        current["Link to standard"] = data[10].find('a')['href']
                        
                    #exception 2 "Non-assessable"
                    else:
                        current["Case number"] = data[3].text.strip()
                        current["Standard violated"] = ""
                        current["Link to standard"] = ""
                        current["Proposed penalty"] = ""
                        current["Amount paid to date"] = ""
                        
                else:
                    current["Case number"] = data[3].text.strip()
                    current["Proposed penalty"] = data[11].text.strip()
                    current["Amount paid to date"] = data[14].text.strip()
                    current["Standard violated"] = data[10].find_all('font')[2].text.strip()
                    current["Link to standard"] = data[10].find('a')['href']

                violations_list.append(current)
    
    #if there's no violation at all   
    else:
        current = {}
        current["Citation number"] = ""
        current["Case number"] = ""
        current["Proposed penalty"] = ""
        current["Amount paid to date"] = ""
        current["Standard violated"] = ""
        current["Link to standard"] = ""
        
        violations_list.append(current)

    df_violation = pd.DataFrame(violations_list)
    filename = violations_MineID+"-violations.csv"
    df_violation.to_csv(filename, index=False)
    print(filename,"successfully saved")

df_new.apply(save_violations_csv,axis=1)


4104757-violations.csv successfully saved
0801306-violations.csv successfully saved
3609931-violations.csv successfully saved


0    None
1    None
2    None
dtype: object